**Problem 1**

In [1]:
import numpy as np
from scipy import linalg

In [2]:
def red_qr(A):
    """ This function implements a modified Gram-Schmidt process
    for reduced QR decomposition of a matrix A.
    """
    m, n = A.shape
    r = np.linalg.matrix_rank(A)
    if r < n:
        print("Inavlid: The matrix A does not have full column rank.")
    Q = np.copy(A)
    R = np.zeros((n, n))
    
    for i in range(n):
        R[i, i] = linalg.norm(Q[:, i])
        Q[:, i] = Q[:, i] / R[i, i]
        for j in range(i + 1, n):
            R[i, j] = np.dot(np.transpose(Q[:, j]), Q[:, i])
            Q[:,j] = Q[:,j] - R[i,j] * Q[:,i]
            
    return Q, R

In [3]:
# Generate a random matrix

A = np.random.random((6,4))
Q, R = red_qr(A)

In [4]:
# Test cases: verify that R is upper triangular, Q orthonormal, QR = A

print("R is upper triangular: ", np.allclose(np.triu(R), R))
print("Q is orthonormal: ", np.allclose(Q.T @ Q, np.identity(4)))
print("A = QR: ", np.allclose(Q @ R, A)) 
Q1, R1 = linalg.qr(A, mode="economic")
print("Q near Q1: ", np.allclose(np.abs(Q), np.abs(Q1)))
print("R near R1: ", np.allclose(np.abs(R), np.abs(R1)))

R is upper triangular:  True
Q is orthonormal:  True
A = QR:  True
Q near Q1:  True
R near R1:  True


**Problem 2**

In [5]:
# Longer version, written to compare the two

def det_qr1(A):
    """Calculate the determinant of A
    """
    
    m, n = A.shape
    r = np.linalg.matrix_rank(A)
    if not n == m:
        print("Invalid: A is not a square matrix.")
    if r < n:
        print("Invalid: A is not an invertible matrix.")
    
    Q, R = linalg.qr(A, mode="economic")
    return abs(np.prod(np.diag(R)))

# Test

# Main function
A = np.random.random((6,6))
r1 = round(det_qr1(A), 4)
print("Rank of A is: ", r1)
r2 = round(abs(linalg.det(A)), 4)
print("Rank of A (scipy QR routine): ", r2)

Rank of A is:  0.0143
Rank of A (scipy QR routine):  0.0143


In [6]:
#Implement the function in one line

det_qr2 = lambda A: abs(np.prod(np.diag(linalg.qr(A, mode='economic')[1])))

# Test

# Single line function
r3 = round(det_qr1(A), 4)
print("Rank of A (single line): ", r3)

Rank of A (single line):  0.0143


**Problem 3**

In [7]:
def linear(A, b):
    """Uses QR decomposition to solve Ax = b
    """
    
    m, n = A.shape
    r = np.linalg.matrix_rank(A)
    length_b = len(b)
    if not n == m:
        print("Invalid: A is not a square matrix.")
    if r < n:
        print("Invalid: A is not an invertible matrix.")
    if not length_b == n:
        print("Invalid: b is not the correct length.")
        
    Q, R = linalg.qr(A, mode = 'economic')
    y = np.dot(np.transpose(Q), b)
    x = np.zeros(n)

    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(R[i, i + 1:], x[i + 1:])) / R[i, i]
    
    return x

In [8]:
# Test

A = np.random.random((4, 4))
b = np.random.random(4)
x1 = linear(A, b)
x2 = np.linalg.solve(A, b)
print("The function 'linear' and 'linalg.solve' yield the "
      "same result:", np.allclose(x1, x2))

The function 'linear' and 'linalg.solve' yield the same result: True


**Problem 4**

In [9]:
def householder(A):
    """Implements the Householder algorithm to compute the QR decomposition of
    a matrix A
    """
    
    m, n = A.shape
    r = np.linalg.matrix_rank(A)
    if r < n:
        print("Inavlid: The matrix A does not have full column rank.")
    
    R = np.copy(A)
    Q = np.eye(m) # The mxm identity matrix
    
    for i in range(n):
        u = np.copy(R[i:, i])
        u[0] = u[0] + sign(u[0]) * linalg.norm(u)
        u = u / linalg.norm(u)
        R[i:, i:] = R[i:, i:] - 2 * np.outer(u, np.dot(np.transpose(u), 
                                                       R[i:, i:]))
        Q[i:, :] = Q[i:, :] - 2 * np.outer(u, np.dot(np.transpose(u), 
                                                     Q[i:, :]))
    
    return np.transpose(Q), R

In [10]:
sign = lambda x: 1 if x >= 0 else -1 # Define the sign function

In [11]:
# Test

A = np.random.random((6, 4))
Q,R = householder(A) 
print("R  is upper triangular: ", np.allclose(np.triu(R), R))
print("Q is orthonormal: ", np.allclose(Q.T @ Q, np.identity(6)))
print("A = QR: ", np.allclose(Q @ R, A))

R  is upper triangular:  True
Q is orthonormal:  True
A = QR:  True


**Problem 5**

In [12]:
def hessenberg(A):
    """Implements the Hessenberg algorithm 
    """
    
    m, n = A.shape
    r = np.linalg.matrix_rank(A)
    if not n == m:
        print("Invalid: A is not a square matrix.")
    if r < n:
        print("Invalid: A is not an invertible matrix.")
        
    H = np.copy(A)
    Q = np.eye(m) # The mxm identity matrix
    
    for i in range(n - 2):
        u = np.copy(H[i + 1:, i])
        u[0] = u[0] + sign(u[0]) * linalg.norm(u)
        u = u / linalg.norm(u)
        H[i + 1:, i:] = H[i + 1:, i:] - 2 * np.outer(u, np.dot(np.transpose(u),
                                                               H[i + 1:, i:]))
        H[:, i + 1:] = H[:, i + 1:] - 2 * np.outer(np.dot(H[:, i + 1:], u), 
                                                   np.transpose(u))
        Q[i + 1:, :] = Q[i + 1:, :] - 2 * np.outer(u, np.dot(np.transpose(u),
                                                             Q[i + 1:, :]))
        
    return H, np.transpose(Q)

In [13]:
# Test

A = np.random.random((6,6))
H, Q = hessenberg(A)
print("Zeroes below the first subdiagonal (matrix H): ", 
      np.allclose(np.triu(H, -1), H))
print("QHQ^T = A: ", np.allclose(Q @ H @ Q.T, A))

Zeroes below the first subdiagonal (matrix H):  True
QHQ^T = A:  True
